This notebook uses the access library to calculate and plot spatial access metrics every census tract in Brazil. 

In [3]:
# Importing necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from access import Access, weights, Datasets
import geopandas as gpd
from shapely import wkt



In [3]:
# Importing data
school_census_df = pd.read_csv("/Users/feliphlvo/Documents/Minerva/Capstone/data/local/school_census.csv",index_col=0)
dem_census_df = pd.read_csv("/Users/feliphlvo/Documents/Minerva/Capstone/data/local/dem_census.csv", index_col=0)

In [4]:
dem_census_df.head()

,sector_id,state,n_households,n_people,avg_monthly_earnings,n_people_15to17_white,n_people_15to17_black,n_people_15to17_asian,n_people_15to17_parda,n_people_15to17_indigenous,...,subdistrict_id,subdistrict_name,neighborhood_name,neighborhood_id,zone,geometry,microregion_id,microregion_name,mesoregion_id,mesoregion_name
0,170025105000002,TO,115.0,300.0,648.06,1.0,0.0,0.0,10.0,0.0,...,1.700251e+10,NaN,NaN,NaN,2.0,"POLYGON ((4316081.043813151 8936951.569070809,...",17003.0,Miracema do Tocantins,1701.0,Ocidental do Tocantins
1,170025105000003,TO,59.0,167.0,433.73,0.0,0.0,0.0,0.0,0.0,...,1.700251e+10,NaN,NaN,NaN,2.0,"POLYGON ((4300023.229366692 8964455.368041243,...",17003.0,Miracema do Tocantins,1701.0,Ocidental do Tocantins
2,170030105000001,TO,286.0,1123.0,927.45,10.0,2.0,3.0,59.0,0.0,...,1.700301e+10,NaN,NaN,NaN,1.0,"POLYGON ((4501590.424471468 9273374.76835, 450...",17001.0,Bico do Papagaio,1701.0,Ocidental do Tocantins
3,170030105000002,TO,294.0,1101.0,478.33,11.0,0.0,1.0,46.0,0.0,...,1.700301e+10,NaN,NaN,NaN,1.0,POLYGON ((4503050.0514419945 9272734.518656082...,17001.0,Bico do Papagaio,1701.0,Ocidental do Tocantins
4,170030105000003,TO,102.0,385.0,453.32,3.0,1.0,0.0,17.0,0.0,...,1.700301e+10,NaN,NaN,NaN,2.0,"POLYGON ((4502995.959709885 9275774.210479755,...",17001.0,Bico do Papagaio,1701.0,Ocidental do Tocantins


In [5]:
print(dem_census_df.columns)
print(school_census_df.columns)

Index(['sector_id', 'state', 'n_households', 'n_people',
       'avg_monthly_earnings', 'n_people_15to17_white',
       'n_people_15to17_black', 'n_people_15to17_asian',
       'n_people_15to17_parda', 'n_people_15to17_indigenous', 'n_people_15',
       'n_people_16', 'n_people_17', 'n_people_15_men', 'n_people_16_men',
       'n_people_17_men', 'n_people_15_women', 'n_people_16_women',
       'n_people_17_women', 'n_people_15to17', 'n_people_15to17_alternative',
       'state_id', 'state_abbrev', 'city_id', 'city_name', 'district_id',
       'district_name', 'subdistrict_id', 'subdistrict_name',
       'neighborhood_name', 'neighborhood_id', 'zone', 'geometry',
       'microregion_id', 'microregion_name', 'mesoregion_id',
       'mesoregion_name'],
      dtype='object')
Index(['region_name', 'region_id', 'state_name', 'state_abbrev', 'state_id',
       'city_name', 'city_id', 'mesoregion_name', 'mesoregion_id',
       'microregion_name', 'microregion_id', 'sector_id', 'school_id',
   

In [6]:
# Only public schools with at least one high school class and regions with at least one high-school aged person
geo_dem_census_df = dem_census_df[dem_census_df["n_people_15to17"] > 0]
geo_school_census_df = school_census_df[(school_census_df["n_classes"] > 0) & (school_census_df["admin_type"] != 4.0)]

In [7]:
# Transforming into geometry
geo_dem_census_df = gpd.GeoDataFrame(geo_dem_census_df, geometry = geo_dem_census_df["geometry"].apply(wkt.loads))
geo_school_census_df = gpd.GeoDataFrame(geo_school_census_df, geometry = geo_school_census_df["geometry"].apply(wkt.loads))




/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/geodataframe.py:31: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(np.asarray(data))


In [8]:
# Importing the distance matric. See Creating Distance Matrix.ipynb
dist_matrix = pd.read_csv("/Users/feliphlvo/Documents/Minerva/Capstone/data/local/dist_matrix.csv", index_col=0)
dist_matrix.head()

,origin,dest,euclidean
0,170025105000002,17010535,13894.563390
1,170030105000001,17004268,1524.518606
2,170030105000001,17002648,10449.349289
3,170030105000002,17004268,1450.603219
4,170030105000002,17002648,10129.682425


In [9]:
# Creating the access object
# See documentation for the Access ibrary here: https://github.com/pysal/access/blob/main/access/access.py
A = Access(
    demand_df=geo_dem_census_df,
    demand_index="sector_id",
    demand_value="n_people_15to17",
    supply_df=geo_school_census_df,
    supply_index="school_id",
    supply_value=["n_classes", "n_teachers"],
    cost_df=dist_matrix,
    cost_origin="origin",
    cost_dest="dest",
    cost_name="euclidean"
    )

In [10]:
# Defining custom weight functions

gaussian = weights.gaussian(5000)   

# Access metrics to calculate
max_cost = 16000
A.two_stage_fca(name="e2sfca", weight_fn=gaussian, cost = "euclidean", max_cost=max_cost)
A.enhanced_two_stage_fca(name="e2sfca", weight_fn=gaussian, cost = "euclidean", max_cost=max_cost)
A.three_stage_fca(name = "3sfca", weight_fn=gaussian, cost = "euclidean", max_cost = max_cost)


access INFO     :: Overwriting e2sfca_n_classes.
access INFO     :: Overwriting e2sfca_n_teachers.


,3sfca_n_classes,3sfca_n_teachers
sector_id,,
110001505000001,0.050560,0.081685
110001505000002,0.049542,0.079882
110001505000003,0.049913,0.080488
110001505000004,0.050614,0.081978
110001505000005,0.048816,0.078745
...,...,...
530010805300152,0.013623,0.000000
530010805300153,0.008719,0.000029
530010805300154,0.008140,0.000030


In [15]:
# Merging access metrics to the main dataframe
access_df = geo_dem_census_df.set_index("sector_id").join(A.access_df, how="left", lsuffix="dem", rsuffix="acc")

In [ ]:
# Convert to SPAR
# TODO

In [16]:
# Saving access metrics to a csv file
access_df.to_csv("/Users/feliphlvo/Documents/Minerva/Capstone/data/local/access_df.csv")